<a href="https://colab.research.google.com/github/rahulsm27/ML/blob/main/Gumbel_Softmax.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import torch
import torch.distributions as D
import torch.nn.functional as F


In [13]:
def sample_gumbel(shape, eps=1e-20):
    U = torch.rand(shape)
    return -torch.log(-torch.log(U + eps) + eps)


def gumbel_softmax_sample(logits, temperature):
    y = logits + sample_gumbel(logits.size())
    return F.softmax(y / temperature, dim=-1)


def gumbel_softmax(logits, temperature, hard=False):
    """
    ST-gumple-softmax
    input: [*, n_class]
    return: flatten --> [*, n_class] an one-hot vector
    """
    y = gumbel_softmax_sample(logits, temperature)

    if not hard:
        return y.view(-1, latent_dim * 2)

    shape = y.size()
    _, ind = y.max(dim=-1)
    y_hard = torch.zeros_like(y).view(-1, shape[-1])
    y_hard.scatter_(1, ind.view(-1, 1), 1)
    y_hard = y_hard.view(*shape)
    # Set gradients w.r.t. y_hard gradients w.r.t. y
    y_hard = (y_hard - y).detach() + y
    return y_hard.view(-1, latent_dim * 2)

In [17]:
temp=1
latent_dim = 10
gumbel_softmax(torch.rand(1, latent_dim*2), temp, False)

tensor([[0.0519, 0.0209, 0.0252, 0.0603, 0.0314, 0.2574, 0.0139, 0.0262, 0.0116,
         0.0094, 0.0260, 0.0074, 0.0202, 0.0070, 0.0186, 0.3177, 0.0272, 0.0232,
         0.0371, 0.0075]])

In [18]:
gumbel_softmax(torch.rand(1, latent_dim*2), temp, True)

tensor([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0.]])